In [ ]:
"""
Yelp Data Collection - Fresh Start
Fetches restaurant data using 3 API keys for different zones
Loads directly into Snowflake Bronze layer
"""

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import requests
import time
from datetime import datetime

# =====================================================
# CONFIGURATION
# =====================================================

YELP_API_KEYS = [
    '',  # For Central Boston
    '',  # For Cambridge/Somerville
    ''   # For Outer Boston
]

SNOWFLAKE_CONFIG = {
    'user': '',
    'password': '',
    'account': '',
    'warehouse': '',
    'database': '',
    'schema': '',
    'role': ''
}

# Geographic zones - each handled by one API key
SEARCH_ZONES = {
    'key1': [  # Central Boston
        {'name': 'Downtown Boston', 'coords': (42.3554, -71.0640), 'radius': 2500},
        {'name': 'North End', 'coords': (42.3647, -71.0542), 'radius': 1500},
        {'name': 'Back Bay', 'coords': (42.3503, -71.0810), 'radius': 2000},
        {'name': 'South End', 'coords': (42.3467, -71.0723), 'radius': 2000},
        {'name': 'Beacon Hill', 'coords': (42.3588, -71.0707), 'radius': 1500},
        {'name': 'Fenway', 'coords': (42.3467, -71.0972), 'radius': 2000},
        {'name': 'Financial District', 'coords': (42.3559, -71.0550), 'radius': 1500},
        {'name': 'Seaport', 'coords': (42.3518, -71.0453), 'radius': 2000},
    ],
    'key2': [  # Cambridge/Somerville
        {'name': 'Harvard Square', 'coords': (42.3736, -71.1197), 'radius': 2000},
        {'name': 'Central Square Cambridge', 'coords': (42.3657, -71.1036), 'radius': 2000},
        {'name': 'Kendall Square', 'coords': (42.3626, -71.0843), 'radius': 2000},
        {'name': 'MIT', 'coords': (42.3601, -71.0942), 'radius': 1500},
        {'name': 'Porter Square', 'coords': (42.3884, -71.1193), 'radius': 1500},
        {'name': 'Inman Square', 'coords': (42.3735, -71.0995), 'radius': 1500},
        {'name': 'Davis Square', 'coords': (42.3967, -71.1226), 'radius': 2000},
        {'name': 'Union Square Somerville', 'coords': (42.3793, -71.0956), 'radius': 2000},
        {'name': 'Assembly Row', 'coords': (42.3925, -71.0770), 'radius': 1500},
    ],
    'key3': [  # Outer Boston
        {'name': 'Allston', 'coords': (42.3534, -71.1312), 'radius': 2500},
        {'name': 'Brighton', 'coords': (42.3482, -71.1605), 'radius': 2500},
        {'name': 'Jamaica Plain', 'coords': (42.3097, -71.1147), 'radius': 2500},
        {'name': 'Roxbury', 'coords': (42.3317, -71.0835), 'radius': 2500},
        {'name': 'Dorchester', 'coords': (42.3011, -71.0661), 'radius': 3000},
        {'name': 'South Boston', 'coords': (42.3331, -71.0476), 'radius': 2500},
        {'name': 'East Boston', 'coords': (42.3717, -71.0395), 'radius': 2500},
        {'name': 'Charlestown', 'coords': (42.3782, -71.0602), 'radius': 2000},
        {'name': 'Brookline', 'coords': (42.3318, -71.1212), 'radius': 2500},
        {'name': 'Watertown', 'coords': (42.3709, -71.1828), 'radius': 2500},
    ]
}

# =====================================================
# YELP API FUNCTIONS
# =====================================================

def fetch_restaurants(api_key, location_name, latitude, longitude, radius, api_key_number):
    """Fetch restaurants from Yelp API"""
    
    headers = {'Authorization': f'Bearer {api_key}'}
    all_restaurants = []
    
    # Yelp allows up to 1000 results per location (20 calls × 50 results)
    # But we'll be conservative and do 12 calls per location
    for offset in range(0, 600, 50):  # 12 calls per location
        params = {
            'latitude': latitude,
            'longitude': longitude,
            'radius': radius,
            'categories': 'restaurants,food',
            'limit': 50,
            'offset': offset
        }
        
        try:
            response = requests.get(
                'https://api.yelp.com/v3/businesses/search',
                headers=headers,
                params=params
            )
            
            if response.status_code == 200:
                data = response.json()
                businesses = data.get('businesses', [])
                
                if not businesses:
                    break  # No more results
                
                # Transform to our format
                for biz in businesses:
                    location = biz.get('location', {})
                    coords = biz.get('coordinates', {})
                    categories = biz.get('categories', [])
                    
                    restaurant = {
                        'restaurant_id': biz.get('id'),
                        'name': biz.get('name'),
                        'phone': biz.get('display_phone', ''),
                        'url': biz.get('url', ''),
                        'address': location.get('address1', ''),
                        'address2': location.get('address2', ''),
                        'city': location.get('city', 'Boston'),
                        'state': location.get('state', 'MA'),
                        'zip_code': location.get('zip_code', ''),
                        'neighborhood': location_name,
                        'full_address': ', '.join([
                            location.get('address1', ''),
                            location.get('city', ''),
                            location.get('state', ''),
                            location.get('zip_code', '')
                        ]),
                        'latitude': coords.get('latitude'),
                        'longitude': coords.get('longitude'),
                        'primary_cuisine': categories[0]['title'] if categories else 'Other',
                        'category_aliases': '|'.join([c['alias'] for c in categories]),
                        'category_titles': '|'.join([c['title'] for c in categories]),
                        'yelp_rating': biz.get('rating'),
                        'yelp_review_count': biz.get('review_count'),
                        'price_tier': biz.get('price', ''),
                        'is_closed': biz.get('is_closed', False),
                        'api_key_used': api_key_number,
                        'search_location': location_name
                    }
                    all_restaurants.append(restaurant)
                
                print(f"  Offset {offset}: Found {len(businesses)} restaurants")
                time.sleep(0.2)  # Rate limiting
                
            else:
                print(f"  Error {response.status_code}: {response.text}")
                break
                
        except Exception as e:
            print(f"  Error: {e}")
            break
    
    return all_restaurants

# =====================================================
# MAIN COLLECTION PROCESS
# =====================================================

def collect_all_data():
    """Main function to collect data from all zones"""
    
    # Connect to Snowflake
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    print("✅ Connected to Snowflake\n")
    
    all_restaurants = []
    
    # Collect from each zone
    for zone_key, locations in SEARCH_ZONES.items():
        api_key_number = int(zone_key[-1])  # Extract 1, 2, or 3
        api_key = YELP_API_KEYS[api_key_number - 1]
        
        print(f"\n{'='*60}")
        print(f"🔑 Using API Key {api_key_number} for {len(locations)} locations")
        print(f"{'='*60}\n")
        
        for location in locations:
            print(f"📍 Searching: {location['name']}...")
            
            restaurants = fetch_restaurants(
                api_key,
                location['name'],
                location['coords'][0],
                location['coords'][1],
                location['radius'],
                api_key_number
            )
            
            all_restaurants.extend(restaurants)
            print(f"  Total so far: {len(all_restaurants)} restaurants\n")
    
    # Convert to DataFrame and remove duplicates
    df = pd.DataFrame(all_restaurants)
    
    print(f"\n{'='*60}")
    print(f"📊 COLLECTION SUMMARY")
    print(f"{'='*60}")
    print(f"Total fetched: {len(df)} restaurants")
    
    df_deduplicated = df.drop_duplicates(subset=['restaurant_id'], keep='first')
    print(f"After deduplication: {len(df_deduplicated)} unique restaurants")
    
    # Prepare DataFrame for Snowflake
    df_deduplicated = df_deduplicated.reset_index(drop=True)
    
    # Convert column names to UPPERCASE to match Snowflake table
    df_deduplicated.columns = df_deduplicated.columns.str.upper()
    
    # Load into Snowflake
    print(f"\n⬆️  Loading into Snowflake Bronze layer...")
    
    success, nchunks, nrows, _ = write_pandas(
        conn,
        df_deduplicated,
        'BRONZE_YELP_RESTAURANTS',
        database='LOCEATS_DB',
        schema='BRONZE',
        auto_create_table=False,
        quote_identifiers=False
    )
    
    print(f"✅ Successfully loaded {nrows} restaurants into Snowflake!")
    
    conn.close()
    return df_deduplicated
# =====================================================
# RUN
# =====================================================

if __name__ == "__main__":
    print("🚀 Starting Yelp Data Collection\n")
    start_time = datetime.now()
    
    df = collect_all_data()
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds() / 60
    
    print(f"\n{'='*60}")
    print(f"✅ COMPLETE!")
    print(f"{'='*60}")
    print(f"Time taken: {duration:.1f} minutes")
    print(f"Total unique restaurants: {len(df)}")
    print(f"\nData loaded into: LOCEATS_DB.BRONZE.BRONZE_YELP_RESTAURANTS")

🚀 Starting Yelp Data Collection

✅ Connected to Snowflake


🔑 Using API Key 1 for 8 locations

📍 Searching: Downtown Boston...
  Offset 0: Found 50 restaurants
  Offset 50: Found 50 restaurants
  Offset 100: Found 50 restaurants
  Offset 150: Found 50 restaurants
  Error 400: {"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 240."}}
  Total so far: 200 restaurants

📍 Searching: North End...
  Offset 0: Found 50 restaurants
  Offset 50: Found 50 restaurants
  Offset 100: Found 50 restaurants
  Offset 150: Found 50 restaurants
  Error 400: {"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 240."}}
  Total so far: 400 restaurants

📍 Searching: Back Bay...
  Offset 0: Found 50 restaurants
  Offset 50: Found 50 restaurants
  Offset 100: Found 50 restaurants
  Offset 150: Found 50 restaurants
  Error 400: {"error": {"code": "VALIDATION_ERROR", "description": "Too many results requ

In [ ]:
"""
Yelp Data Collection - Enhanced Version
Fetches MORE restaurant data with granular neighborhoods
Skips existing restaurants to avoid duplicates
Loads directly into Snowflake Bronze layer
"""

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import requests
import time
from datetime import datetime

# =====================================================
# CONFIGURATION
# =====================================================

YELP_API_KEYS = [
    '',  # For Central Boston
    '',  # For Cambridge/Somerville
    ''   # For Outer Boston
]

SNOWFLAKE_CONFIG = {
    'user': '',
    'password': '',
    'account': '',
    'warehouse': '',
    'database': '',
    'schema': '',
    'role': ''
}

# Geographic zones - MORE GRANULAR for better coverage
SEARCH_ZONES = {
    1: [  # Central Boston - API Key 1 (20 locations)
        {'name': 'Downtown Boston', 'lat': 42.3554, 'lon': -71.0640, 'radius': 1500},
        {'name': 'North End', 'lat': 42.3647, 'lon': -71.0542, 'radius': 1000},
        {'name': 'Back Bay East', 'lat': 42.3503, 'lon': -71.0810, 'radius': 1500},
        {'name': 'Back Bay West', 'lat': 42.3486, 'lon': -71.0890, 'radius': 1500},
        {'name': 'South End North', 'lat': 42.3467, 'lon': -71.0723, 'radius': 1500},
        {'name': 'South End South', 'lat': 42.3420, 'lon': -71.0730, 'radius': 1500},
        {'name': 'Beacon Hill', 'lat': 42.3588, 'lon': -71.0707, 'radius': 1000},
        {'name': 'Fenway East', 'lat': 42.3467, 'lon': -71.0972, 'radius': 1500},
        {'name': 'Fenway West', 'lat': 42.3450, 'lon': -71.1050, 'radius': 1500},
        {'name': 'Financial District', 'lat': 42.3559, 'lon': -71.0550, 'radius': 1000},
        {'name': 'Seaport East', 'lat': 42.3518, 'lon': -71.0453, 'radius': 1500},
        {'name': 'Seaport West', 'lat': 42.3490, 'lon': -71.0380, 'radius': 1500},
        {'name': 'Chinatown', 'lat': 42.3513, 'lon': -71.0624, 'radius': 800},
        {'name': 'Theater District', 'lat': 42.3519, 'lon': -71.0643, 'radius': 800},
        {'name': 'West End', 'lat': 42.3645, 'lon': -71.0661, 'radius': 1000},
        {'name': 'Leather District', 'lat': 42.3517, 'lon': -71.0537, 'radius': 800},
        {'name': 'Bay Village', 'lat': 42.3495, 'lon': -71.0700, 'radius': 800},
        {'name': 'Newbury Street', 'lat': 42.3505, 'lon': -71.0845, 'radius': 1000},
        {'name': 'Copley Square', 'lat': 42.3495, 'lon': -71.0770, 'radius': 800},
        {'name': 'Prudential Center', 'lat': 42.3467, 'lon': -71.0818, 'radius': 800},
    ],
    2: [  # Cambridge/Somerville - API Key 2 (18 locations)
        {'name': 'Harvard Square', 'lat': 42.3736, 'lon': -71.1197, 'radius': 1500},
        {'name': 'Harvard North', 'lat': 42.3800, 'lon': -71.1200, 'radius': 1000},
        {'name': 'Central Square Cambridge', 'lat': 42.3657, 'lon': -71.1036, 'radius': 1500},
        {'name': 'Kendall Square', 'lat': 42.3626, 'lon': -71.0843, 'radius': 1500},
        {'name': 'MIT', 'lat': 42.3601, 'lon': -71.0942, 'radius': 1000},
        {'name': 'Porter Square', 'lat': 42.3884, 'lon': -71.1193, 'radius': 1200},
        {'name': 'Inman Square', 'lat': 42.3735, 'lon': -71.0995, 'radius': 1200},
        {'name': 'Davis Square', 'lat': 42.3967, 'lon': -71.1226, 'radius': 1500},
        {'name': 'Union Square Somerville', 'lat': 42.3793, 'lon': -71.0956, 'radius': 1500},
        {'name': 'Assembly Row', 'lat': 42.3925, 'lon': -71.0770, 'radius': 1200},
        {'name': 'East Cambridge', 'lat': 42.3675, 'lon': -71.0820, 'radius': 1200},
        {'name': 'North Cambridge', 'lat': 42.3890, 'lon': -71.1280, 'radius': 1500},
        {'name': 'Cambridgeport', 'lat': 42.3615, 'lon': -71.1055, 'radius': 1200},
        {'name': 'Teele Square', 'lat': 42.4000, 'lon': -71.1185, 'radius': 1000},
        {'name': 'Ball Square', 'lat': 42.3995, 'lon': -71.1110, 'radius': 1000},
        {'name': 'Magoun Square', 'lat': 42.3990, 'lon': -71.1096, 'radius': 1000},
        {'name': 'Winter Hill', 'lat': 42.3950, 'lon': -71.0970, 'radius': 1200},
        {'name': 'Lechmere', 'lat': 42.3705, 'lon': -71.0763, 'radius': 1000},
    ],
    3: [  # Outer Boston - API Key 3 (22 locations)
        {'name': 'Allston North', 'lat': 42.3550, 'lon': -71.1312, 'radius': 1800},
        {'name': 'Allston South', 'lat': 42.3520, 'lon': -71.1330, 'radius': 1800},
        {'name': 'Brighton Center', 'lat': 42.3482, 'lon': -71.1605, 'radius': 1800},
        {'name': 'Cleveland Circle', 'lat': 42.3365, 'lon': -71.1495, 'radius': 1500},
        {'name': 'Jamaica Plain Center', 'lat': 42.3097, 'lon': -71.1147, 'radius': 1800},
        {'name': 'Jamaica Plain South', 'lat': 42.3000, 'lon': -71.1100, 'radius': 1800},
        {'name': 'Roxbury', 'lat': 42.3317, 'lon': -71.0835, 'radius': 1800},
        {'name': 'Nubian Square', 'lat': 42.3308, 'lon': -71.0830, 'radius': 1000},
        {'name': 'Dorchester North', 'lat': 42.3011, 'lon': -71.0661, 'radius': 2000},
        {'name': 'Dorchester South', 'lat': 42.2850, 'lon': -71.0650, 'radius': 2000},
        {'name': 'South Boston', 'lat': 42.3331, 'lon': -71.0476, 'radius': 1800},
        {'name': 'Andrew Square', 'lat': 42.3302, 'lon': -71.0574, 'radius': 1200},
        {'name': 'East Boston', 'lat': 42.3717, 'lon': -71.0395, 'radius': 1800},
        {'name': 'Maverick Square', 'lat': 42.3693, 'lon': -71.0393, 'radius': 1000},
        {'name': 'Charlestown', 'lat': 42.3782, 'lon': -71.0602, 'radius': 1500},
        {'name': 'Brookline Village', 'lat': 42.3318, 'lon': -71.1212, 'radius': 1500},
        {'name': 'Coolidge Corner', 'lat': 42.3420, 'lon': -71.1210, 'radius': 1200},
        {'name': 'Watertown', 'lat': 42.3709, 'lon': -71.1828, 'radius': 1800},
        {'name': 'Newton Centre', 'lat': 42.3295, 'lon': -71.1925, 'radius': 1500},
        {'name': 'Roslindale', 'lat': 42.2851, 'lon': -71.1270, 'radius': 1800},
        {'name': 'West Roxbury', 'lat': 42.2790, 'lon': -71.1600, 'radius': 1800},
        {'name': 'Hyde Park', 'lat': 42.2545, 'lon': -71.1247, 'radius': 1800},
    ]
}

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def fetch_restaurants(api_key, location_name, latitude, longitude, radius, api_key_number, existing_ids):
    """Fetch restaurants from Yelp API, skipping ones we already have"""
    
    headers = {'Authorization': f'Bearer {api_key}'}
    all_restaurants = []
    new_count = 0
    duplicate_count = 0
    
    # Fetch up to 240 results (5 calls × 50 or until we run out)
    for offset in range(0, 250, 50):
        params = {
            'latitude': latitude,
            'longitude': longitude,
            'radius': radius,
            'categories': 'restaurants,food',
            'limit': 50,
            'offset': offset
        }
        
        try:
            response = requests.get(
                'https://api.yelp.com/v3/businesses/search',
                headers=headers,
                params=params
            )
            
            if response.status_code == 200:
                data = response.json()
                businesses = data.get('businesses', [])
                
                if not businesses:
                    break  # No more results
                
                # Transform to our format
                for biz in businesses:
                    biz_id = biz.get('id')
                    
                    # Skip if we already have this restaurant
                    if biz_id in existing_ids:
                        duplicate_count += 1
                        continue
                    
                    location = biz.get('location', {})
                    coords = biz.get('coordinates', {})
                    categories = biz.get('categories', [])
                    
                    # Build full address safely
                    addr_parts = []
                    if location.get('address1'):
                        addr_parts.append(location.get('address1'))
                    if location.get('city'):
                        addr_parts.append(location.get('city'))
                    if location.get('state'):
                        addr_parts.append(location.get('state'))
                    if location.get('zip_code'):
                        addr_parts.append(location.get('zip_code'))
                    
                    restaurant = {
                        'restaurant_id': biz_id,
                        'name': biz.get('name'),
                        'phone': biz.get('display_phone', ''),
                        'url': biz.get('url', ''),
                        'address': location.get('address1', ''),
                        'address2': location.get('address2', ''),
                        'city': location.get('city', 'Boston'),
                        'state': location.get('state', 'MA'),
                        'zip_code': location.get('zip_code', ''),
                        'neighborhood': location_name,
                        'full_address': ', '.join(filter(None, addr_parts)),
                        'latitude': coords.get('latitude'),
                        'longitude': coords.get('longitude'),
                        'primary_cuisine': categories[0]['title'] if categories else 'Other',
                        'category_aliases': '|'.join([c['alias'] for c in categories]),
                        'category_titles': '|'.join([c['title'] for c in categories]),
                        'yelp_rating': biz.get('rating'),
                        'yelp_review_count': biz.get('review_count'),
                        'price_tier': biz.get('price', ''),
                        'is_closed': biz.get('is_closed', False),
                        'api_key_used': api_key_number,
                        'search_location': location_name
                    }
                    all_restaurants.append(restaurant)
                    existing_ids.add(biz_id)  # Add to set so we don't fetch again
                    new_count += 1
                
                time.sleep(0.2)  # Rate limiting
                
            elif response.status_code == 400:
                # Hit the 240 limit, that's ok
                break
            elif response.status_code == 429:
                print(f"    ⚠️  Rate limit hit")
                break
            else:
                print(f"    Error {response.status_code}")
                break
                
        except Exception as e:
            print(f"    Error: {e}")
            break
    
    if new_count > 0 or duplicate_count > 0:
        print(f"    Found {new_count} new, skipped {duplicate_count} duplicates")
    
    return all_restaurants

# =====================================================
# MAIN COLLECTION PROCESS
# =====================================================

def collect_all_data():
    """Main function to collect data from all zones"""
    
    # Connect to Snowflake
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    print("✅ Connected to Snowflake\n")
    
    # Get existing restaurant IDs to avoid duplicates
    cursor = conn.cursor()
    cursor.execute("SELECT RESTAURANT_ID FROM BRONZE_YELP_RESTAURANTS")
    existing_ids = set(row[0] for row in cursor.fetchall())
    print(f"📊 Found {len(existing_ids)} existing restaurants in database")
    print(f"🎯 Will only fetch NEW restaurants\n")
    
    all_new_restaurants = []
    
    # Collect from each zone
    for zone_key, locations in SEARCH_ZONES.items():
        api_key_number = zone_key
        api_key = YELP_API_KEYS[api_key_number - 1]
        
        print(f"\n{'='*60}")
        print(f"🔑 Using API Key {api_key_number} for {len(locations)} locations")
        print(f"{'='*60}\n")
        
        for location in locations:
            print(f"📍 Searching: {location['name']}...")
            
            restaurants = fetch_restaurants(
                api_key,
                location['name'],
                location['lat'],
                location['lon'],
                location['radius'],
                api_key_number,
                existing_ids
            )
            
            all_new_restaurants.extend(restaurants)
    
    print(f"\n{'='*60}")
    print(f"📊 COLLECTION SUMMARY")
    print(f"{'='*60}")
    print(f"New restaurants found: {len(all_new_restaurants)}")
    print(f"Total in database (existing + new): {len(existing_ids)}")
    
    if len(all_new_restaurants) == 0:
        print("\n⚠️  No new restaurants found. Database already has good coverage!")
        conn.close()
        return None
    
    # Convert to DataFrame
    df = pd.DataFrame(all_new_restaurants)
    
    # Prepare DataFrame for Snowflake
    df = df.reset_index(drop=True)
    df.columns = df.columns.str.upper()
    
    # Load into Snowflake
    print(f"\n⬆️  Loading {len(df)} new restaurants into Snowflake...")
    
    success, nchunks, nrows, _ = write_pandas(
        conn,
        df,
        'BRONZE_YELP_RESTAURANTS',
        database='LOCEATS_DB',
        schema='BRONZE',
        auto_create_table=False,
        quote_identifiers=False
    )
    
    print(f"✅ Successfully loaded {nrows} new restaurants into Snowflake!")
    
    conn.close()
    return df

# =====================================================
# RUN
# =====================================================

if __name__ == "__main__":
    print("🚀 Starting Enhanced Yelp Data Collection\n")
    start_time = datetime.now()
    
    df = collect_all_data()
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds() / 60
    
    print(f"\n{'='*60}")
    print(f"✅ COMPLETE!")
    print(f"{'='*60}")
    print(f"Time taken: {duration:.1f} minutes")
    if df is not None:
        print(f"New restaurants added: {len(df)}")
    print(f"\nData in: LOCEATS_DB.BRONZE.BRONZE_YELP_RESTAURANTS")
    print(f"\nVerify with: SELECT COUNT(*) FROM BRONZE_YELP_RESTAURANTS;")

🚀 Starting Enhanced Yelp Data Collection

✅ Connected to Snowflake

📊 Found 1706 existing restaurants in database
🎯 Will only fetch NEW restaurants


🔑 Using API Key 1 for 20 locations

📍 Searching: Downtown Boston...
    Found 11 new, skipped 189 duplicates
📍 Searching: North End...
    Found 15 new, skipped 185 duplicates
📍 Searching: Back Bay East...
    Found 4 new, skipped 196 duplicates
📍 Searching: Back Bay West...
    Found 18 new, skipped 182 duplicates
📍 Searching: South End North...
    Found 5 new, skipped 195 duplicates
📍 Searching: South End South...
    Found 12 new, skipped 188 duplicates
📍 Searching: Beacon Hill...
    Found 4 new, skipped 196 duplicates
📍 Searching: Fenway East...
    Found 9 new, skipped 191 duplicates
📍 Searching: Fenway West...
    Found 13 new, skipped 187 duplicates
📍 Searching: Financial District...
    Found 16 new, skipped 184 duplicates
📍 Searching: Seaport East...
    Found 7 new, skipped 193 duplicates
📍 Searching: Seaport West...
    Found